In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [35]:
folder = "../../Data/hpc_out/rwm_ma_3_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/rwm_op_2_6r7m/"

id_index = 1
separator = "_"
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

acceptance_rates = Vector{Float64}([])
min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        try
            @load "$folder$file" input output
            # Crude test to exclude runs that remained dispersed
            if maximum(output["Es"]) - minimum(output["Es"]) < 10.0# || input["T"] > 1.5
                println("E_diff = $(maximum(output["Es"]) - minimum(output["Es"]))")
                continue
            end

            # # Hacky selection of a subset of results
            if input["T"] != 4.0 continue end
            
            id = parse(Int, split(file, separator)[id_index])
            exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
            exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
            theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
            string = "$(id): $(input["T"]) | $(input["overlap_slope"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
            push!(acceptance_rates, output["αs"][end])
            evaluation_strings[id] = string
            min_Es[id] = minimum(output["Es"])
        catch e
            println(e)
            println("Error in $file")
        end
    end
end

In [36]:
println("Minimal configuration id: $(argmin(min_Es)) | Average acceptance rate: $(sum(acceptance_rates) / length(acceptance_rates))")

Minimal configuration id: 87 | Average acceptance rate: 0.41854661573534424


In [37]:
for elem in evaluation_strings
    println(elem)
end



3: 4.0 | 1.0 | 0.6872012 | 13144.11229548463, 13073.918459747012, 13077.773074498962 | 29.760428919324436

5: 4.0 | 1.0 | 0.55513513 | 13144.112295375311, 13086.165889485652, 13144.11229530796 | 71.69728103712906
6: 4.0 | 1.0 | 0.36014348 | 13144.112295402245, 13056.342247332012, 13065.64054090672 | 110.15124969788569
7: 4.0 | 1.0 | 0.7516563 | 13144.112295372373, 13087.718583397538, 13098.243721779529 | 140.52233639805684
8: 4.0 | 1.0 | 0.19462925 | 13364.238044140811, 13000.105530849694, 13076.896624967601 | 14.21683323804214
9: 4.0 | 1.0 | 0.33697537 | 13144.112295850366, 13047.392677034393, 13049.781147379867 | 31.096046321848316
10: 4.0 | 1.0 | 0.14727543 | 13144.112295081204, 13028.688454116118, 13077.763205592015 | 20.06197334779124
11: 4.0 | 1.0 | 0.45104435 | 13144.1122954525, 12993.722369664709, 12993.722369664709 | 10.803833677788306
12: 4.0 | 1.0 | 0.22121596 | 13743.955556865469, 13035.271764173258, 13144.112295488803 | 19.65688814865544
13: 4.0 | 1.0 | 0.36287522 | 1314

In [38]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [ ]:
@load "../../Data/hpc_out/rwm_ma_2_6r7m/81_rwm_wip_2_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [55]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "../../Data/tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end



# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
